# Midterm Project: Machine Learning Modeling
## 一、数据加载与准备

In [2]:
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet , SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import re
import warnings

# 加载数据
train_df = pd.read_csv("C:/Users/86158/Desktop/Mid_Test/Data/ruc_Class25Q1_train.csv")
test_df = pd.read_csv("C:/Users/86158/Desktop/Mid_Test/Data/ruc_Class25Q1_test.csv")
detail_df = pd.read_csv("C:/Users/86158/Desktop/Mid_Test/Data/ruc_Class25Q1_details.csv")

warnings.filterwarnings('ignore') 

## 二、数据预处理
### 2.1 数据检查

In [3]:
print("训练数据列名:", train_df.columns.tolist())
print("详情数据列名:", detail_df.columns.tolist())

训练数据列名: ['城市', '区域', '板块', '环线', '小区名称', '价格', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份']
详情数据列名: ['区县', '名称', '城市', '板块', '环线位置', '小区地址', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y']


### 2.2 预处理函数构建（特征工程）

In [25]:
def improved_preprocessing(df, is_train=True, target_enc=None):
    # 记录初始行数
    initial_rows = len(df)
    print(f"\n{'训练数据' if is_train else '测试数据'}初始行数: {initial_rows}")
    
    processed_df = df.copy()
    
    # 1. 合并小区详情数据
    if '小区名称' in processed_df.columns and '名称' in detail_df.columns:
        print("\n正在合并小区详情数据...")
        print(f"详情数据初始行数: {len(detail_df)}")
        
        detail_df_clean = detail_df.drop_duplicates(subset=['名称'])
        print(f"去重后详情数据行数: {len(detail_df_clean)}")
        
        processed_df = pd.merge(
            processed_df, 
            detail_df_clean, 
            left_on='小区名称', 
            right_on='名称', 
            how='left'
        )
        print(f"合并后行数: {len(processed_df)} (应与合并前相同)")
        
        # 删除合并后多余的列
        if '名称' in processed_df.columns:
            processed_df = processed_df.drop('名称', axis=1)
    
    # 2. 删除无用和泄露特征
    drop_cols = [
        '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行',
        'lon', 'lat', '小区名称',
        '房屋年限', '上次交易', '年份',
        '梯户比例', '交易权属',
        '房屋户型', '所在楼层'
    ]
    # 只删除实际存在的列
    drop_cols = [col for col in drop_cols if col in processed_df.columns]
    print(f"\n将删除以下列: {drop_cols}")
    processed_df = processed_df.drop(drop_cols, axis=1)
    print(f"删除列后行数: {len(processed_df)} (应与删除前相同)")
    
    # 3. 处理日期特征
    if '交易时间' in processed_df.columns:
        try:
            processed_df['交易时间'] = pd.to_datetime(processed_df['交易时间'], errors='coerce')
            processed_df['交易年份'] = processed_df['交易时间'].dt.year
            processed_df['交易月份'] = processed_df['交易时间'].dt.month
            processed_df = processed_df.drop('交易时间', axis=1)
            print("\n已处理日期特征")
        except Exception as e:
            print("日期处理错误:", e)
    
    # 4. 数值特征处理
    # 建筑面积处理
    if '建筑面积' in processed_df.columns:
        try:
            processed_df['建筑面积'] = processed_df['建筑面积'].astype(str).str.replace('㎡', '').astype(float)
            processed_df['建筑面积_ori'] = processed_df['建筑面积']
            print("\n已处理建筑面积特征")
        except Exception as e:
            print("建筑面积处理错误:", e)
            processed_df['建筑面积'] = processed_df['建筑面积'].astype(float, errors='ignore')
            processed_df['建筑面积_ori'] = processed_df['建筑面积']
    
    # 5. 特征工程
    # 建筑结构
    if '建筑结构' in processed_df.columns:
        processed_df['建筑结构_钢混'] = processed_df['建筑结构'].apply(lambda x: 1 if str(x) == '钢混结构' else 0)
        processed_df['建筑结构_混合'] = processed_df['建筑结构'].apply(lambda x: 1 if str(x) == '混合结构' else 0)
        print("\n已处理建筑结构特征")
    
    # 装修情况
    if '装修情况' in processed_df.columns:
        processed_df['装修_精装'] = processed_df['装修情况'].apply(lambda x: 1 if str(x) == '精装' else 0)
        processed_df['装修_简装'] = processed_df['装修情况'].apply(lambda x: 1 if str(x) == '简装' else 0)
        print("\n已处理装修情况特征")
    
    # 电梯信息
    if '配备电梯' in processed_df.columns:
        processed_df['有电梯'] = processed_df['配备电梯'].map({'有':1, '无':0}).fillna(0)
        print("\n已处理电梯信息特征")
    
    # 6. 创建交互特征
    if '建筑面积' in processed_df.columns and '室数' in processed_df.columns:
        processed_df['面积_室数'] = processed_df['建筑面积'] * processed_df['室数']
        print("\n已创建交互特征: 面积×室数")
    
    # 7. 高基数分类特征编码
    high_card_cols = []
    for col in ['区域', '板块']:
        if col in processed_df.columns:
            high_card_cols.append(col)
    
    if high_card_cols:
        print(f"\n正在处理高基数分类特征: {high_card_cols}")
        if is_train and '价格' in processed_df.columns and '建筑面积_ori' in processed_df.columns:
            try:
                # 使用频率编码
                processed_df['log_price_per_area'] = np.log1p(processed_df['价格'] / processed_df['建筑面积_ori'])
                
                for col in high_card_cols:
                    freq = processed_df.groupby(col)['log_price_per_area'].mean().to_dict()
                    processed_df[col+'_encoded'] = processed_df[col].map(freq)
                    # 保存编码器状态
                    if target_enc is None:
                        target_enc = {}
                    target_enc[col] = freq
                
                # 删除原始分类列
                processed_df = processed_df.drop(high_card_cols, axis=1)
                print(f"已完成分类特征编码，删除了原始列: {high_card_cols}")
            except Exception as e:
                print("目标编码错误:", e)
        elif not is_train and target_enc is not None:
            # 测试集应用编码
            try:
                for col in high_card_cols:
                    if col in target_enc:
                        processed_df[col+'_encoded'] = processed_df[col].map(target_enc[col])
                # 删除原始分类列
                processed_df = processed_df.drop(high_card_cols, axis=1)
                print(f"已应用预训练的编码器，删除了原始列: {high_card_cols}")
            except Exception as e:
                print("测试集编码错误:", e)
    
    
    # 最终行数检查
    final_rows = len(processed_df)
    print(f"\n预处理完成，最终行数: {final_rows}")
    if initial_rows != final_rows:
        print(f"警告: 行数发生变化 (初始: {initial_rows}, 最终: {final_rows})")
    else:
        print("行数保持一致")
    
    return processed_df, target_enc

### 2.3 执行数据预处理

In [26]:
try:
    print("\n=== 预处理训练数据 ===")
    train_processed, target_encoder = improved_preprocessing(train_df, is_train=True)
    
    print("\n=== 预处理测试数据 ===")
    test_processed, _ = improved_preprocessing(test_df, is_train=False, target_enc=target_encoder)
    
    # 检查结果
    if 'train_processed' in locals() and 'test_processed' in locals():
        print("\n预处理成功完成！")
        print("训练数据形状:", train_processed.shape)
        print("测试数据形状:", test_processed.shape)
    else:
        print("\n预处理未能完成")
except Exception as e:
    print("\n执行预处理时出错:", str(e))


=== 预处理训练数据 ===

训练数据初始行数: 84133

正在合并小区详情数据...
详情数据初始行数: 3100
去重后详情数据行数: 2973
合并后行数: 84133 (应与合并前相同)

将删除以下列: ['房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '小区名称', '房屋年限', '上次交易', '年份', '梯户比例', '交易权属', '房屋户型', '所在楼层']
删除列后行数: 84133 (应与删除前相同)

已处理日期特征

已处理建筑面积特征

已处理装修情况特征

已处理电梯信息特征

正在处理高基数分类特征: ['区域']
已完成分类特征编码，删除了原始列: ['区域']

预处理完成，最终行数: 84133
行数保持一致

=== 预处理测试数据 ===

测试数据初始行数: 14786

正在合并小区详情数据...
详情数据初始行数: 3100
去重后详情数据行数: 2973
合并后行数: 14786 (应与合并前相同)

将删除以下列: ['房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '小区名称', '房屋年限', '上次交易', '年份', '梯户比例', '交易权属', '房屋户型', '所在楼层']
删除列后行数: 14786 (应与删除前相同)

已处理日期特征

已处理建筑面积特征

已处理装修情况特征

已处理电梯信息特征

正在处理高基数分类特征: ['区域']
已应用预训练的编码器，删除了原始列: ['区域']

预处理完成，最终行数: 14786
行数保持一致

预处理成功完成！
训练数据形状: (84133, 48)
测试数据形状: (14786, 47)


### 2.4 异常值处理

In [44]:
X = train_processed.drop(['价格', 'log_price_per_area', '建筑面积_ori'], axis=1, errors='ignore')
y = train_processed['log_price_per_area']

# 异常值处理
print("\n=== 异常值处理 ===")
numeric_cols = X.select_dtypes(include=np.number).columns

for col in numeric_cols:
    if X[col].nunique() > 10:  # 只对连续型数值特征处理
        lower = X[col].quantile(0.01)
        upper = X[col].quantile(0.99)
        print(f"{col}: 截断范围 [{lower:.2f}, {upper:.2f}]")
        
        # 保存原始值计数用于报告
        original_count = len(X)
        outliers_low = (X[col] < lower).sum()
        outliers_high = (X[col] > upper).sum()
        
        # 执行截断
        X[col] = X[col].clip(lower, upper)
        
        # 打印处理结果
        print(f"  处理了 {outliers_low + outliers_high} 个异常值 "
              f"(低于1%: {outliers_low}, 高于99%: {outliers_high})")



=== 异常值处理 ===
板块_x: 截断范围 [27.00, 807.00]
  处理了 1256 个异常值 (低于1%: 775, 高于99%: 481)
建筑面积: 截断范围 [30.18, 279.61]
  处理了 1684 个异常值 (低于1%: 842, 高于99%: 842)
区县: 截断范围 [3.00, 99.97]
  处理了 1427 个异常值 (低于1%: 642, 高于99%: 785)
板块_y: 截断范围 [27.00, 807.00]
  处理了 1197 个异常值 (低于1%: 751, 高于99%: 446)
容 积 率: 截断范围 [0.73, 8.30]
  处理了 1383 个异常值 (低于1%: 745, 高于99%: 638)
停车位: 截断范围 [3.00, 6000.00]
  处理了 811 个异常值 (低于1%: 409, 高于99%: 402)
coord_x: 截断范围 [106.30, 126.71]
  处理了 1541 个异常值 (低于1%: 765, 高于99%: 776)
coord_y: 截断范围 [29.43, 45.89]
  处理了 1075 个异常值 (低于1%: 670, 高于99%: 405)
交易月份: 截断范围 [1.00, 12.00]
  处理了 0 个异常值 (低于1%: 0, 高于99%: 0)
区域_encoded: 截断范围 [8.25, 11.34]
  处理了 1159 个异常值 (低于1%: 344, 高于99%: 815)


### 2.5 缺失值处理

In [45]:
print("\n=== 缺失值处理 ===")
print("处理前缺失值统计:")
print(X.isna().sum().sort_values(ascending=False))

# 数值型缺失值
for col in numeric_cols:
    if X[col].isna().any():
        median_val = X[col].median()
        X[col] = X[col].fillna(median_val if not np.isnan(median_val) else 0)

# 类别型缺失值
categorical_cols = X.select_dtypes(include='object').columns
for col in categorical_cols:
    if X[col].isna().any():
        X[col] = X[col].fillna('missing')

print("\n处理后缺失值统计:")
print(X.isna().sum().sort_values(ascending=False))

y_price = train_processed['价格']  # 用于价格级别评估


=== 缺失值处理 ===
处理前缺失值统计:
抵押信息          84133
别墅类型          83384
套内面积          58987
物业办公电话        56272
环线位置          42758
环线            41407
供热费           37973
供暖            27920
开发商           13328
物业公司          12914
停车位           12096
停车费用          10016
容 积 率          8821
绿 化 率          8646
配备电梯           8315
燃气费            7777
建筑年代           7325
物 业 费          7220
供水             6957
供电             6592
建筑结构_y         6294
板块_y           5795
区县             5729
楼栋总数           5677
小区地址           5677
城市_y           5677
房屋总数           5677
物业类别           5677
产权描述           5677
coord_x        5677
coord_y        5677
建筑结构_x          605
装修情况            605
房屋用途              2
城市_x              0
板块_x              0
建筑面积              0
产权所属              0
房屋朝向              0
交易年份              0
交易月份              0
装修_精装             0
装修_简装             0
有电梯               0
区域_encoded        0
dtype: int64

处理后缺失值统计:
城市_x          0
板块_x          0
环线            0
建筑面

## 三、模型构建与训练

### 3.1 预处理管道构建

In [35]:
numeric_features = X_train.select_dtypes(include=np.number).columns
categorical_features = X_train.select_dtypes(include='object').columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

### 3.2 模型配置

In [36]:
models = {
    'OLS': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', LinearRegression(n_jobs=-1))
    ]),
    'Lasso': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', SGDRegressor(
            penalty='l1',
            alpha=0.0001,
            max_iter=2000,
            tol=1e-3,
            random_state=111))
    ]),
    'Ridge': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', SGDRegressor(
            penalty='l2',
            alpha=0.0001,
            max_iter=2000,
            tol=1e-3,
            random_state=111))
    ]),
    'ElasticNet': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', SGDRegressor(
            penalty='elasticnet',
            alpha=0.0001,
            l1_ratio=0.5,
            max_iter=2000,
            tol=1e-3,
            random_state=111))
    ])
}

## 四、模型评估与结果分析

### 4.1 模型训练与评估

In [37]:
results = {}
for name, pipe in models.items():
    print(f"\n训练 {name}...")
    try:
        pipe.fit(X_train, y_train)
        
        # 常规评估
        y_pred = pipe.predict(X_val)
        price_pred = np.exp(y_pred) * X_val['建筑面积'] if '建筑面积' in X_val.columns else np.exp(y_pred)
        price_true = np.exp(y_val) * X_val['建筑面积'] if '建筑面积' in X_val.columns else np.exp(y_val)
        
        # 6折交叉验证（在训练集上）
        cv_scores = cross_val_score(
            pipe, X_train, y_train,
            cv=6,
            scoring='neg_mean_absolute_error',
            n_jobs=-1
        )
        
        results[name] = {
            'In-sample MAE': mean_absolute_error(y_train, pipe.predict(X_train)),
            'Out-sample MAE': mean_absolute_error(y_val, y_pred),
            'Price MAE': mean_absolute_error(price_true, price_pred),
            'Price RMSE': np.sqrt(mean_squared_error(price_true, price_pred)),
            'Cross-validation MAE': -cv_scores.mean()  # 交叉验证结果
        }
    except Exception as e:
        print(f"{name}训练失败:", str(e))
        results[name] = {k: np.nan for k in ['In-sample MAE', 'Out-sample MAE', 'Price MAE', 'Price RMSE', 'Cross-validation MAE']}



训练 OLS...

训练 Lasso...

训练 Ridge...

训练 ElasticNet...


### 4.2 结果展示与分析

In [39]:
metrics = pd.DataFrame(results).T

# 确定最佳模型
valid_results = metrics.dropna()
if not valid_results.empty:
    best_model = valid_results['Cross-validation MAE'].idxmin()
    metrics.loc['Best Model'] = metrics.loc[best_model]
    
print("\n=== 测试结果 ===")
print(metrics[['In-sample MAE', 'Out-sample MAE', 'Cross-validation MAE', 'Price MAE', 'Price RMSE']])




=== 测试结果 ===
            In-sample MAE  Out-sample MAE  Cross-validation MAE  \
OLS              0.090271        0.110282              0.112245   
Lasso            0.149035        0.151241              0.151512   
Ridge            0.130874        0.133988              0.134815   
ElasticNet       0.140959        0.143379              0.144701   
Best Model       0.090271        0.110282              0.112245   

                Price MAE    Price RMSE  
OLS         224799.017005  8.858480e+05  
Lasso       327440.864845  9.569172e+05  
Ridge       297732.763145  1.030084e+06  
ElasticNet  309469.195884  9.303855e+05  
Best Model  224799.017005  8.858480e+05  


## 五、预测结果生成

### 5.1 测试集预测准备

In [40]:
# 确保测试数据包含所有训练特征
missing_cols = set(X_train.columns) - set(test_processed.columns)
for col in missing_cols:
    test_processed[col] = 0  # 用0填充缺失特征

# 确保列顺序一致
X_test = test_processed[X_train.columns]

### 5.2 执行预测并保存结果

In [42]:
# 生成所有模型的预测结果
predictions = {}
for model_name, model in models.items():
    try:
        test_log_pred = model.predict(X_test)
        
        # 转换回原始价格
        if '建筑面积_ori' in test_processed.columns:
            test_price_pred = np.exp(test_log_pred) * test_processed['建筑面积_ori']
        else:
            test_price_pred = np.exp(test_log_pred)
        
        # 保存预测结果
        predictions[model_name] = test_price_pred
        
        # 创建DataFrame并保存
        submission = pd.DataFrame({
            'ID': test_df['ID'],
            'Price': np.round(test_price_pred).astype(int)
        })
        submission.to_csv(f"prediction_{model_name}.csv", index=False)
        
    except Exception as e:
        print(f"模型 {model_name} 预测失败: {str(e)}")
        predictions[model_name] = None

# 输出预测统计信息
print("\n所有模型预测结果统计:")
for model_name, pred in predictions.items():
    if pred is not None:
        print(f"{model_name}: 预测值范围 [{pred.min():.0f}, {pred.max():.0f}]")
    else:
        print(f"{model_name}: 无有效预测结果")


所有模型预测结果统计:
OLS: 预测值范围 [84695, 61574625]
Lasso: 预测值范围 [108582, 37661625]
Ridge: 预测值范围 [119738, 27689734]
ElasticNet: 预测值范围 [114408, 39328853]


### 5.3 最终结果

In [ ]:
metrics = pd.DataFrame(results).T
metrics['Datahub Score'] = [78.422, 73.782, 74.505, 74.798]  # 按OLS,Lasso,Ridge,ElasticNet顺序

print("\n=== 最终结果 ===")
# 确定最佳模型（基于Cross-validation MAE）
valid_results = metrics.dropna()
if not valid_results.empty:
    best_model = valid_results['Cross-validation MAE'].idxmin()
    metrics.loc['Best Model'] = metrics.loc[best_model]
    metrics.at['Best Model', 'Datahub Score'] = metrics.loc[best_model, 'Datahub Score']

# 输出表格（保留3位小数）
final_table = metrics[['In-sample MAE', 'Out-sample MAE', 'Cross-validation MAE', 'Datahub Score']]
final_table.columns = ['In sample', 'Out of sample', 'Cross-validation', 'Datahub Score']
print("\n| Metrics        | In sample | Out of sample | Cross-validation | Datahub Score |")
print("|----------------|-----------|---------------|-------------------|---------------|")
for model in final_table.index:
    print(f"| {model:14} | {final_table.loc[model, 'In sample']:>9.3f} | "
          f"{final_table.loc[model, 'Out of sample']:>13.3f} | "
          f"{final_table.loc[model, 'Cross-validation']:>17.3f} | "
          f"{final_table.loc[model, 'Datahub Score']:>13.3f} |")


=== 最终结果 ===

| Metrics        | In sample | Out of sample | Cross-validation | Datahub Score |
|----------------|-----------|---------------|-------------------|---------------|
| OLS            |     0.090 |         0.110 |             0.112 |        78.422 |
| Lasso          |     0.149 |         0.151 |             0.152 |        73.782 |
| Ridge          |     0.131 |         0.134 |             0.135 |        74.505 |
| ElasticNet     |     0.141 |         0.143 |             0.145 |        74.798 |
| Best Model     |     0.090 |         0.110 |             0.112 |        78.422 |
